# Disambiguating geographic names and the usefullness of a JSON structured database for storing and processing data from APIs 

In [1]:
import idigbio # Provides wrapper functions to query the iDigBio API
from pymongo import MongoClient # Provides connection to MongoDB
import pandas as pd # Allow us to work with data structured as tables

from pymongo import ASCENDING
import requests
import uuid
import json
import os
from datetime import datetime

iDigBio API results can be rendered in JSON format (structured nested object type) or as a flat table using Pandas. In this ocassion we are going to be using the JSON format for most of the tasks.

Create a handler for your queries, so that they will be returned in JSON format

In [2]:
idigbio_json = idigbio.json()

The query format to pass the idigbio API is similar to JSON in this case the most closely format which is a Python dictionary. Create an object with the query specifying a bounding box  with coordinates that include US and British Virgin Islands.

In [89]:
rq = {
    # Necessary if we are going to loop querying the API until we have all the documents needed
    "uuid": {
        "type": "range",
        "gt": "000000000-0000-0000-0000-000000000000",
        "lte": "ffffffff-ffff-ffff-ffff-ffffffffffff"
    },
    # Bounding box coordinates for the area of the Virgin Islands
    "geopoint": {
        "type": "geo_bounding_box",
        "top_left": {
            "lat": 18.971233956586723,
            "lon": -65.137939453125
        },
        "bottom_right": {
            "lat": 17.368988699356095,
            "lon": -64.1436767578125
        }
    }
}

Set extra parameters to pass with the query to the API

In [90]:
sort = [ { "uuid": "asc"}] # Sorts all returened values by uuid from low values to high values
limit=1 # Limit the number of records returned
offset = 0 # Set how many records to skip of the results

The response from the API is a JSON document which in converted to a Python dictionary. This makes it so that it is already in the format needed to be stored in a document based database as MongoDB which we will be using to store and further process the data

In [91]:
# Setup the connection to the database
client = MongoClient('localhost', 27017)
db = client['idigbio_demo']
occ_col = db['idigbio_occurrences_demo'] # This last entry will be our handler to the occurrence collection

The JSON format allow us to have increadible flexibility in how to organize our data. It can have any level of nestedness for a single entry (document). Think of this nestedness of having multiple tables to JOIN when using SQL.

In [92]:
def full_record_build(item, download_date, download_uuid, query, query_type=None):
    
    
    full_record = {
            'idigbio_uuid':item['uuid'],
            'download_history':[
                {
                    'download_date': download_date,
                    'query_type': query_type,
                    'query': query,
                    'download_uuid': download_uuid
                }
            ],
            'idigbio_record': {
                "download_uuid": download_uuid,
                "occurrence_data": item
            }
    }
    
    
    return full_record

Initial feed for the database

In [93]:
api_output = idigbio_json.search_records(rq, sort=sort, limit=limit, offset=offset)

In [94]:
api_output.keys()

dict_keys(['itemCount', 'lastModified', 'items', 'attribution'])

In [12]:
download_date = datetime.today()
download_uuid = str(uuid.uuid1())

In [13]:
for item in api_output['items']:
    
    full_record = full_record_build(item, download_date, download_uuid, query=rq, query_type='bounding_box')
    
    occ_col.insert_one(full_record)

In [14]:
# Finds a single record in the database and returns a Python dictionary 
occ_col.find_one()

{'_id': ObjectId('603680e8373937d10e567d14'),
 'idigbio_uuid': '0000012b-9bb8-42f4-ad3b-c958cb22ae45',
 'download_history': [{'download_date': datetime.datetime(2021, 2, 24, 11, 37, 57, 937000),
   'query_type': 'bounding_box',
   'query': {'uuid': {'type': 'range',
     'gt': '000000000-0000-0000-0000-000000000000',
     'lte': 'ffffffff-ffff-ffff-ffff-ffffffffffff'},
    'geopoint': {'type': 'geo_bounding_box',
     'top_left': {'lat': 18.971233956586723, 'lon': -65.137939453125},
     'bottom_right': {'lat': 17.368988699356095, 'lon': -64.1436767578125}}},
   'download_uuid': 'a6d4d442-76be-11eb-8789-54271e0868e7'}],
 'idigbio_record': {'download_uuid': 'a6d4d442-76be-11eb-8789-54271e0868e7',
  'occurrence_data': {'uuid': '0000012b-9bb8-42f4-ad3b-c958cb22ae45',
   'type': 'records',
   'etag': 'bc37b04afb6c493e99b2ca641d9aee0a46d5ec1e',
   'data': {},
   'indexTerms': {'startdayofyear': 233,
    'continent': 'north america',
    'country': 'virgin islands, u.s.',
    'collectioncode

In [15]:
# Create index on the idigbio_uuid field
occ_col.create_index(
    [("idigbio_uuid", ASCENDING)],
    unique=True,
    name='idigbio_uuid_idx'
)

'idigbio_uuid_idx'

In [16]:
def idigbio_insertdb(full_record,collection_obj, update=False):
    
    query = {
        'idigbio_uuid': full_record['idigbio_uuid']
    }
    
    if collection_obj.find_one(query):
        if update:
            update = {
                '$push':{'download_history': full_record['download_history'][0]}
            }
            collection_obj.find_one_and_update(query, update)
    else:
        collection_obj.insert_one(full_record) 

In [17]:
def search_idigbio_updatedb(rq, sort, limit, offset, collection, query_type=None, update=False):
    
    search_count = idigbio_json.count_records(rq)
    
    if search_count:
        print(search_count)

        download_date = datetime.today()
        download_uuid = str(uuid.uuid1())
        
        # iDigBio Api seems to retrieve reliably up to 5000 records at a time
        if search_count > 5000:
            
            rq_orig = rq

            # Inspired from code by wilsotc (https://github.com/iDigBio/idigbio-search-api/issues/32)
            while True:
                
                partial_search = idigbio_json.search_records(rq, sort=sort, limit=limit, offset=offset)
                
                print(partial_search['itemCount'])
                
                for item in partial_search['items']:
                    # Create full record
                    full_record = full_record_build(item, download_date, download_uuid, query=rq_orig, query_type=query_type)
                    # Insert record to db
                    idigbio_insertdb(full_record, occ_col, update=update)
                    
                if partial_search['itemCount'] < limit:
                    break
                else:
                    rq['uuid']['gt'] = partial_search['items'][-1]['uuid']

                    
        else:
            complete_search = idigbio_json.search_records(rq,sort=sort)
            for item in complete_search['items']:
                full_record = full_record_build(item, download_date, download_uuid, query=rq, query_type=query_type)
                idigbio_insertdb(full_record,occ_col, update=update)
    else:
        print('No Results found')

In [19]:
%%time
#CPU times: user 38.7 s, sys: 4.46 s, total: 43.2 s
#Wall time: 3min 56s
search_idigbio_updatedb(rq, sort=sort, limit=5000, offset=0, collection=occ_col, query_type='bounding_box', update=False)

38395
38395
33395
28395
23395
18395
13395
8395
3395
CPU times: user 38.7 s, sys: 4.46 s, total: 43.2 s
Wall time: 3min 56s


In [20]:
occ_col.count_documents({})

38395

Let's check what are the country codes of the results

In [21]:
query = {}
field = 'idigbio_record.occurrence_data.indexTerms.countrycode'
occ_col.distinct(field, query)

['aia',
 'bhs',
 'dom',
 'gbr',
 'jam',
 'mar',
 'phl',
 'pri',
 'tto',
 'umi',
 'usa',
 'ven',
 'vgb',
 'vir']

How many records have country codes actually available?

In [22]:
query = {
    'idigbio_record.occurrence_data.indexTerms.countrycode': {'$exists': True}
}
occ_col.count_documents(query)

13153

In [24]:
# Percentage of records with country code
13153/38395*100

34.257064721969

In [25]:
query = {}
field = 'idigbio_record.occurrence_data.indexTerms.country'
occ_col.distinct(field, query)

['anguilla',
 'antilles',
 'bahamas',
 'british virgin is',
 'british virgin is.',
 'british virgin island/s',
 'british virgin islands',
 'british virgin islands (british)',
 'british west indies',
 'british/u.s. virgin islands',
 'caribbean',
 'danish west indies',
 'dominican republic',
 'estados unidos de america',
 "etats-unis d'amérique",
 'great britain',
 'guana (bvi)',
 'guiana',
 'indeterminado',
 'jamaica',
 'lesser antilles',
 'morocco',
 'netherlands antilles',
 'peter island (bvi)',
 'philippines',
 'puerto rico',
 'saint croix; u.s. virgin islands',
 'saint thomas; u.s. virgin islands',
 'st. croix',
 'trinidad and tobago',
 'u. s. virgin islands',
 'u.s. virgin is.',
 'u.s. virgin island',
 'u.s.virgin islands',
 'united kingdom',
 'united kingdom overseas territory',
 'united states',
 'united states minor outlying islands',
 'united states virgin islands',
 'us virgin is',
 'us virgin island/s',
 'us virgin islands',
 'venezuela',
 'vierges(i.)',
 'virgin islands',
 '

In [26]:
def db_unique_geo_name(field, collection):
    pipeline = [
        {
            '$group': {
                '_id': field, 
                'total': {
                    '$sum': 1
                }
            }
        },
        {
            '$sort':{
                'total':-1
            }
        }
    ]
    cursor = collection.aggregate(pipeline)
    return [ (i['_id'], i['total']) for i in list(cursor) ]

In [27]:
db_unique_geo_name('$idigbio_record.occurrence_data.indexTerms.country', occ_col)

[('virgin islands, u.s.', 8222),
 ('british virgin islands', 7257),
 ('virgin islands (usa)', 5605),
 ('virgin islands', 5327),
 ('united states', 3975),
 ('virgin islands (u.s.)', 2138),
 (None, 1572),
 ('virgin islands (british)', 505),
 ('united kingdom', 464),
 ('caribbean', 412),
 ('west indies', 332),
 ('virgin islands, british', 258),
 ('united states virgin islands', 256),
 ('virgin islands (uk)', 232),
 ('virgin islands (us)', 221),
 ('british virgin island/s', 219),
 ('puerto rico', 207),
 ('us virgin island/s', 202),
 ('îles vierges des états-unis', 163),
 ('guana (bvi)', 134),
 ('us virgin is', 121),
 ('british virgin is.', 91),
 ('virgin islands (u.k. or u.s.)', 77),
 ('virgin islands [us]', 73),
 ('virgin islands of the usa', 46),
 ('us virgin islands', 46),
 ('virgin islands, w indies', 39),
 ('british virgin is', 34),
 ('lesser antilles', 26),
 ('vierges(i.)', 24),
 ('antilles', 16),
 ('u.s.virgin islands', 12),
 ('estados unidos de america', 11),
 ('united kingdom over

Unique Country Names

In [30]:
unique_country_names = [
    ('virgin islands, u.s.', 8222),
    ('british virgin islands', 7257),
    ('virgin islands (usa)', 5605),
    ('virgin islands', 5327),
    ('virgin islands (u.s.)', 2138),
    ('virgin islands (british)', 505),
    ('virgin islands, british', 258),
    ('united states virgin islands', 256),
    ('virgin islands (uk)', 232),
    ('virgin islands (us)', 221),
    ('british virgin island/s', 219),
    ('us virgin island/s', 202),
    ('îles vierges des états-unis', 163),
    ('guana (bvi)', 134),
    ('us virgin is', 121),
    ('british virgin is.', 91),
    ('virgin islands (u.k. or u.s.)', 77),
    ('virgin islands [us]', 73),
    ('us virgin islands', 46),
    ('virgin islands of the usa', 46),
    ('virgin islands, w indies', 39),
    ('british virgin is', 34),
    ('u.s.virgin islands', 12),
    ('united kingdom overseas territory', 8),
    ('saint croix; u.s. virgin islands', 7),
    ('u.s. virgin is.', 5),
    ('saint thomas; u.s. virgin islands', 5),
    ('united states minor outlying islands', 4),
    ('îles vierges britanniques', 4),
    ("etats-unis d'amérique", 3),
    ('peter island (bvi)', 3),
    ('st. croix', 2),
    ('british/u.s. virgin islands', 1),
    ('british virgin islands (british)', 1),
    ('british west indies', 1),
    ('u.s. virgin island', 1),
    ('u. s. virgin islands', 1)
]

In [29]:
db_unique_geo_name('$idigbio_record.occurrence_data.indexTerms.stateprovince', occ_col)

[(None, 17804),
 ('st. croix', 5039),
 ('saint john', 3891),
 ('virgin islands', 2271),
 ('guana island', 1363),
 ('virgin islands of the united states', 1242),
 ('st. john', 1172),
 ('saint croix', 916),
 ('saint thomas', 601),
 ('st. thomas', 547),
 ('british virgin islands', 388),
 ('st.croix', 387),
 ('anegada', 272),
 ('st. croix island', 197),
 ('st. john island', 166),
 ('(îles vierges des états-unis)', 162),
 ('st john island', 160),
 ('us virgin islands', 154),
 ('tortola', 141),
 ('virgin gorda', 129),
 ('st croix island', 122),
 ('st thomas', 107),
 ('tortola island', 103),
 ('norman island', 88),
 ('st. johns', 86),
 ('u.s. virgin islands', 81),
 ('guana', 67),
 ('arecibo', 66),
 ('st croix', 62),
 ('st. thomas island', 53),
 ('west indies', 49),
 ('anegada island', 43),
 ('st john', 41),
 ('saint croix island', 40),
 ('st.thomas', 35),
 ('u. s. virgin islands', 27),
 ('vierges(i.)', 24),
 ('virigin islands', 22),
 ('jost van dyke', 20),
 ('puerto rico', 16),
 ('saint thoma

In [ ]:
Unique stateprovince names

In [31]:
unique_stateprovince_names = [
    ('st. croix', 5039),
    ('saint john', 3891),
    ('virgin islands', 2271),
    ('guana island', 1363),
    ('virgin islands of the united states', 1242),
    ('st. john', 1172),
    ('saint croix', 916),
    ('saint thomas', 601),
    ('st. thomas', 547),
    ('british virgin islands', 388),
    ('st.croix', 387),
    ('anegada', 272),
    ('st. croix island', 197),
    ('st. john island', 166),
    ('(îles vierges des états-unis)', 162),
    ('st john island', 160),
    ('us virgin islands', 154),
    ('tortola', 141),
    ('virgin gorda', 129),
    ('st croix island', 122),
    ('st thomas', 107),
    ('tortola island', 103),
    ('norman island', 88),
    ('st. johns', 86),
    ('u.s. virgin islands', 81),
    ('guana', 67),
    ('st croix', 62),
    ('st. thomas island', 53),
    ('anegada island', 43),
    ('st john', 41),
    ('saint croix island', 40),
    ('st.thomas', 35),
    ('u. s. virgin islands', 27),
    ('virigin islands', 22),
    ('jost van dyke', 20),
    ('puerto rico', 16),
    ('saint thomas island', 16),
    ('united states virgin islands', 14),
    ('st.thomas(i.)', 10),
    ('jost van dyke island', 9),
    ('virgin islands [us]', 7),
    ('saint john island', 7),
    ('(îles vierges britanniques)', 4),
    ('st thomas island', 4),
    ('virgin gorda island', 4),
    ('british virgin ids.', 4),
    ('u s virgin ids.', 3),
    ('sanit croix', 1),
    ('st .croix', 1),
    ('us virgin islands (usvi)', 1)
]

In [32]:
db_unique_geo_name('$idigbio_record.occurrence_data.indexTerms.county', occ_col)

[(None, 35031),
 ('none', 1029),
 ('saint croix', 645),
 ('saint thomas', 522),
 ('saint john', 339),
 ('st. croix', 154),
 ('saint croix island', 107),
 ('st. thomas', 94),
 ('east end', 65),
 ('tortola', 60),
 ('st. john', 58),
 ('northside a quarter', 23),
 ('central', 22),
 ('cruz bay', 21),
 ('west end quarter', 21),
 ('west end', 18),
 ('northwest', 13),
 ('mona island', 12),
 ('british virgin islands', 12),
 ('christiansted', 10),
 ('st. john island', 10),
 ('coral bay', 9),
 ('little north side', 9),
 ('virgin gorda', 8),
 ('st. john distr.', 8),
 ('frederiksted', 7),
 ('south side', 6),
 ('sion farm', 5),
 ('northside', 5),
 ('little saba island', 5),
 ('prince quarter', 4),
 ('san german', 4),
 ('reef bay', 4),
 ('water island', 3),
 ('northcentral', 3),
 ('southside', 3),
 ('southwest', 3),
 ('anegada', 3),
 ('charlotte amalie', 3),
 ('[not stated]', 2),
 ('anegada island', 2),
 ('east end b quarter', 2),
 ('company quarter', 2),
 ('washington', 2),
 ('st. croix island', 2),

Unique county names

In [33]:
unique_county_names = [
    ('saint croix', 645),
    ('saint thomas', 522),
    ('saint john', 339),
    ('st. croix', 154),
    ('saint croix island', 107),
    ('st. thomas', 94),
    ('tortola', 60),
    ('st. john', 58),
    ('british virgin islands', 12),
    ('st. john island', 10),
    ('virgin gorda', 8),
    ('anegada', 3),
    ('anegada island', 2),
    ('st. croix island', 2),
    ("st. john's island", 2),
    ('saint johns island', 1),
    ('saint thomas island', 1),
    ('saint johns', 1)
]

In [34]:
all_names = unique_country_names + unique_stateprovince_names + unique_county_names

In [35]:
# get unique entries of the name variations
all_names_unique = [i[0] for i in all_names]
all_names_unique = list(set(all_names_unique))

In [36]:
len(all_names_unique)

83

In [45]:
rq_country = {
    "uuid": {
        "type": "range",
        "gt": "000000000-0000-0000-0000-000000000000",
        "lte": "ffffffff-ffff-ffff-ffff-ffffffffffff"
    },
    "country": all_names_unique,
    "kingdom": "plantae"
    
}
rq_stateprovince = {
    "uuid": {
        "type": "range",
        "gt": "000000000-0000-0000-0000-000000000000",
        "lte": "ffffffff-ffff-ffff-ffff-ffffffffffff"
    },
    "stateprovince": all_names_unique,
    "kingdom": "plantae"
    
}

rq_county = {
    "uuid": {
        "type": "range",
        "gt": "000000000-0000-0000-0000-000000000000",
        "lte": "ffffffff-ffff-ffff-ffff-ffffffffffff"
    },
    "county": all_names_unique,
    "kingdom": "plantae"
    
}

In [41]:
sort = [ { "uuid": "asc"} ]
limit= 5000
offset = 0

In [39]:
%%time
#CPU times: user 2min 21s, sys: 13.6 s, total: 2min 34s
#Wall time: 11min 29s
# n_records: 145488
search_idigbio_updatedb(
    rq_country, # country query
    sort=sort,
    limit=limit,
    offset=offset,
    collection=occ_col,
    query_type='country_search',
    update=True)

145488
145488
140488
135488
130488
125488
120488
115488
110488
105488
100488
95488
90488
85488
80488
75488
70488
65488
60488
55488
50488
45488
40488
35488
30488
25488
20488
15488
10488
5488
488
CPU times: user 2min 21s, sys: 13.6 s, total: 2min 34s
Wall time: 11min 29s


In [44]:
%%time
#CPU times: user 51.9 s, sys: 4.89 s, total: 56.8 s
#Wall time: 4min 32s
search_idigbio_updatedb(
    rq_stateprovince,
    sort=sort,
    limit=limit,
    offset=offset,
    collection=occ_col,
    query_type='stateprovince_search',
    update=True)

50506
50506
45506
40506
35506
30506
25506
20506
15506
10506
5506
506
CPU times: user 51.9 s, sys: 4.89 s, total: 56.8 s
Wall time: 4min 32s


In [46]:
%%time
#CPU times: user 11.7 s, sys: 1.26 s, total: 12.9 s
#Wall time: 1min 13s
search_idigbio_updatedb(
    rq_county,
    sort=sort,
    limit=limit,
    offset=offset,
    collection=occ_col,
    query_type='county_search',
    update=True)

11248
11248
6248
1248
CPU times: user 11.7 s, sys: 1.26 s, total: 12.9 s
Wall time: 1min 13s


In [49]:
query = {'idigbio_record.occurrence_data.indexTerms.kingdom':'plantae'}
occ_col.count_documents(query)

195654

Let's query the database and group the information from the country, stateprovince and county fields and with that create a set of unique combinations based on those fields to pass to a geocoding API.

In [50]:
pipeline = [
    {
        '$match': {
            'idigbio_record.occurrence_data.indexTerms.kingdom': 'plantae'
        }
    }, {
        '$project': {
            '_id': 0, 
            'idigbio_uuid': 1, 
            'country': '$idigbio_record.occurrence_data.indexTerms.country', 
            'stateprovince': '$idigbio_record.occurrence_data.indexTerms.stateprovince', 
            'county': '$idigbio_record.occurrence_data.indexTerms.county'
        }
    }, {
        '$group': {
            '_id': {
                'country': '$country', 
                'stateprovince': '$stateprovince', 
                'county': '$county'
            }, 
            'idigbio_uuids': {
                '$push': '$idigbio_uuid'
            }, 
            'total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'total': -1
        }
    }
]

In [51]:
cursor = occ_col.aggregate(pipeline)

In [52]:
df = pd.DataFrame.from_records(cursor)

In [53]:
df.head()

,_id,idigbio_uuids,total
0,{'country': 'puerto rico'},"[4d33c74e-f87b-4ec9-aca1-f819dbe48c3c, 82f26f9...",75853
1,"{'country': 'caribbean', 'stateprovince': 'pue...","[0003b658-3bd1-4a45-9322-659077a95c35, 00065c8...",20176
2,"{'country': 'virgin islands, u.s.'}","[00aabdf5-2ed8-4089-b075-329dfc225c8d, 00f1873...",9165
3,"{'country': 'united states', 'stateprovince': ...","[b8f31f55-6e09-432b-9446-cf7b0ce1c03e, 00013d0...",7158
4,"{'country': 'jamaica', 'stateprovince': 'saint...","[002db79c-bc70-476f-99d9-0f51260a79a9, 00401f1...",3229


In [54]:
df.shape

(1370, 3)

In [55]:
df['geostring'] = df._id.apply(lambda x: f"{x.get('country','')};{x.get('stateprovince','')};{x.get('county','')}")

In [56]:
df.head()

,_id,idigbio_uuids,total,geostring
0,{'country': 'puerto rico'},"[4d33c74e-f87b-4ec9-aca1-f819dbe48c3c, 82f26f9...",75853,puerto rico;;
1,"{'country': 'caribbean', 'stateprovince': 'pue...","[0003b658-3bd1-4a45-9322-659077a95c35, 00065c8...",20176,caribbean;puerto rico;
2,"{'country': 'virgin islands, u.s.'}","[00aabdf5-2ed8-4089-b075-329dfc225c8d, 00f1873...",9165,"virgin islands, u.s.;;"
3,"{'country': 'united states', 'stateprovince': ...","[b8f31f55-6e09-432b-9446-cf7b0ce1c03e, 00013d0...",7158,united states;puerto rico;
4,"{'country': 'jamaica', 'stateprovince': 'saint...","[002db79c-bc70-476f-99d9-0f51260a79a9, 00401f1...",3229,jamaica;saint thomas;


In [57]:
df_records = df.to_dict('records')

In [60]:
df_records[100]

{'_id': {'country': 'virgin islands, u.s.',
  'stateprovince': 'saint john island'},
 'idigbio_uuids': ['00bf3528-126a-444d-a8d5-d05a45deec5c',
  '06e11637-e029-4bbc-9709-92cc8c29ace8',
  '0858a2a0-5d95-4a38-8c58-f258a6b60814',
  '0aa6b4b7-8223-42dd-bc97-a91a304afba5',
  '0c01d830-ff0e-4631-a43d-6731ce1b73f7',
  '0fe2de34-9d6c-48a2-b0cb-634cdd4bddc6',
  '1056d0b2-e7ba-43bb-aa7b-8c8dbcbb8281',
  '12f53e80-c17f-4e55-89c4-91795a09d7c2',
  '151c5dc9-f4db-4364-a285-c525c710029e',
  '16904fe5-8155-4f95-972b-416915f5db42',
  '18e56122-2721-44ef-915c-1f9e7b9cc3c3',
  '1cef3e42-7dd7-4fd8-a966-d7a38580c3fa',
  '22fb7ed4-f40a-473b-bc07-774f8c991715',
  '23954729-306d-41a1-94c6-881e6ddc42b4',
  '24200d60-25be-4b6d-b8c8-3f6c56f4b0de',
  '25aba7d1-9440-42ca-a8e4-04a9c35f4225',
  '27cdb066-d9f9-405d-a571-bab75226b84a',
  '2ba54924-b61a-4466-becf-c881751d5017',
  '2c2be522-fb5b-4663-b1f9-06d5e112d1ab',
  '2c84208d-df1d-4c10-9f30-297cc8b3d276',
  '2c904351-7ac6-4cd9-abb5-6a87d008bb76',
  '2cca08ae-d946

Pass one example combination to the Geonames API

In [95]:
params = {
    'q': df_records[100]['geostring'],
    'featureClass':'A',
    'operator':'AND',
    'username':os.environ.get('GEONAMES_USER'),
    'fuzzy':1
}
r = requests.get('http://api.geonames.org/searchJSON?', params=params)

In [96]:
res_json = r.json()

In [68]:
res_json

{'totalResultsCount': 64,
 'geonames': [{'adminCode1': '020',
   'lng': '-64.73771',
   'geonameId': 7267903,
   'toponymName': 'Saint John Island',
   'countryId': '4796775',
   'fcl': 'A',
   'population': 4170,
   'countryCode': 'VI',
   'name': 'Saint John Island',
   'fclName': 'country, state, region,...',
   'adminCodes1': {'ISO3166_2': 'J'},
   'countryName': 'U.S. Virgin Islands',
   'fcodeName': 'first-order administrative division',
   'adminName1': 'Saint John Island',
   'lat': '18.32758',
   'fcode': 'ADM1'},
  {'adminCode1': '020',
   'lng': '-64.7389',
   'geonameId': 9252024,
   'toponymName': 'Central',
   'countryId': '4796775',
   'fcl': 'A',
   'population': 0,
   'countryCode': 'VI',
   'name': 'Central',
   'fclName': 'country, state, region,...',
   'adminCodes1': {'ISO3166_2': 'J'},
   'countryName': 'U.S. Virgin Islands',
   'fcodeName': 'second-order administrative division',
   'adminName1': 'Saint John Island',
   'lat': '18.3415',
   'fcode': 'ADM2'},
  {'

In [69]:
set([i['countryCode'] for i in res_json['geonames']])

{'VI'}

Now let's do it for all combinations

In [72]:
%%time
#CPU times: user 8.64 s, sys: 1.02 s, total: 9.66 s
#Wall time: 9min
count = 0
for record in df_records:
    
    
    params = {
        'q':record['geostring'],
        'featureClass':'A',
        'fuzzy':1,
    }
    base_params = params.copy()
    params['username'] = os.environ.get('GEONAMES_USER')
    
    
    r = requests.get('http://api.geonames.org/searchJSON?', params=params)
    
    
    try:
        json_res = r.json()
        if r.status_code == 200 and json_res['totalResultsCount'] > 0:

            
            processing = {
                'geostring_combination': record['geostring'],
                'combintation_type':'country_stateprovince_county',
                'geonames_query': base_params,
                'geonames_count': json_res['totalResultsCount'],
                'geonames_results': json_res['geonames'],
                'matched_countries': list(set([i['countryCode'] for i in json_res['geonames']]))
            }
            
            # Search and update records in the database
            db_query = {'idigbio_uuid':{'$in':record['idigbio_uuids']}}
            update = {
                '$set':{'idigbio_record.processing':processing}
            }
            occ_col.update_many(db_query, update)
            
            
    except Exception as e:
        print(record['geostring'], e)
        
    count= count+1
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


Retrieve records that matched VG and VI (British and US Vrigin Islands)

In [73]:
pipeline = [
    {
        '$match': {
            'idigbio_record.occurrence_data.indexTerms.kingdom': 'plantae', 
            'idigbio_record.processing.matched_countries': {
                '$in': [
                    'VG', 'VI'
                ]
            }
        }
    }, {
        '$project': {
            'idigbio_uuid': 1, 
            '_id': 0, 
            'matched_countries': '$idigbio_record.processing.matched_countries', 
            'geostring': '$idigbio_record.processing.geostring_combination'
        }
    }
]

In [97]:
cursor = occ_col.aggregate(pipeline)

In [98]:
df_virgin_islands = pd.DataFrame.from_records(cursor)

In [99]:
df_virgin_islands.shape

(20099, 3)

In [77]:
df_virgin_islands

,idigbio_uuid,matched_countries,geostring
0,0000012b-9bb8-42f4-ad3b-c958cb22ae45,[VI],"virgin islands, u.s.;saint john;"
1,0028a171-91b9-4596-a0c4-933675d072ba,"[VI, PR]",united states;virgin islands;
2,00329d44-7789-45f3-a80d-8f00369db5b5,"[VI, PR]",united states;virgin islands;
3,0052f18b-55f2-48c2-9ad0-68ed396c336f,[VI],"virgin islands, u.s.;st. john island;"
4,00684881-5ab2-4d36-a7a8-658568380dad,[VI],"virgin islands, u.s.;saint thomas;"
...,...,...,...
20094,6eebfcca-f858-4b0c-a86b-94de32a00f25,"[US, VI]",united states;;saint johns
20095,7a6943f4-de23-47bf-8603-48fb500765c4,"[US, VI]",united states;;saint croix
20096,d3647124-7ac7-4b4c-83cd-82fcd910843b,"[US, VI]",united states;;saint johns
20097,db6054ee-cf91-4a5c-ac0f-f9f187185a75,"[DM, KN, JE, US, MT, SC, AG, GD, VI, JM, LR, C...",;;saint john


In [84]:
query= {'idigbio_uuid':"00329d44-7789-45f3-a80d-8f00369db5b5"}
data = occ_col.find_one(query)

In [85]:
data

{'_id': ObjectId('60368171373937d10e567d31'),
 'idigbio_uuid': '00329d44-7789-45f3-a80d-8f00369db5b5',
 'download_history': [{'download_date': datetime.datetime(2021, 2, 24, 11, 39, 50, 712000),
   'query_type': 'bounding_box',
   'query': {'uuid': {'type': 'range',
     'gt': '000000000-0000-0000-0000-000000000000',
     'lte': 'ffffffff-ffff-ffff-ffff-ffffffffffff'},
    'geopoint': {'type': 'geo_bounding_box',
     'top_left': {'lat': 18.971233956586723, 'lon': -65.137939453125},
     'bottom_right': {'lat': 17.368988699356095, 'lon': -64.1436767578125}}},
   'download_uuid': 'ea071784-76be-11eb-8789-54271e0868e7'},
  {'download_date': datetime.datetime(2021, 2, 24, 12, 3, 32, 316000),
   'query_type': 'stateprovince_search',
   'query': {'uuid': {'type': 'range',
     'gt': '000000000-0000-0000-0000-000000000000',
     'lte': 'ffffffff-ffff-ffff-ffff-ffffffffffff'},
    'stateprovince': ['virgin islands (u.k. or u.s.)',
     'virgin islands (us)',
     'virgin islands (british)',
 